In [3]:
import pandas as pd
import numpy as np
import random
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
random.seed(21)

from typing_extensions import typing

In [10]:
path1 = "/content/gdrive/MyDrive/NGO/Devolution at 10 survey Sep.xlsx"
path2 = "./Devolution at 10 survey Sep.xlsx"
data = pd.read_excel(path2)
data.head()

,29. State the 3 main challenges facing your county government.,Theme 1: Corruption/poor transparency/accountability/,Theme 2: Poor Civic Education/Public Participation/GRMs access to Information,Theme 3: Weak oversight/ Governance/procurement process,Theme 4: Conflicts between executive and county assembly,Theme 5: Poor legislative capacity,Theme 6: Weak finance management/poor leadership/procurement process,Theme 7: Climate issues/ insecurities Conflict,Theme 8: Human resource (HR) issues/ Nepotism/Cronyism/ tribalism,Theme 9: Poor service delivery/Lack of Digitization of services,"Theme 10: Inadequate resources, delayed treasury disbursements",Theme 11: Lack of opportunities /innovation/Investment unemployement/ Poverty/,Theme 12: Poor Inclusion/Gender/PWDs/Support to youths,Theme 13: Poor County systems/operations/ OSR collection,Theme 14: Poor InterGovernmental relations,Theme 15: Lack of political good will/ Political Influence/ Bad politics,Theme 16: Lack of Technical Capacity,Theme 17: Huge wage bill/Pending bills,Theme 18: Insecurity,"Theme 19: Poor Partnerships(Private Sector/NGOs), Poor Coordination"
0,No CBEF \nNo openness in procurement process b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"1.Nepotism,corruption and favouritism in emplo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Lack of guidelines on complaints and complimen...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Corruption, unemployment, poverty",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Water, waste management, wetlands encroachment...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Data cleaning and preparation from Part A

# Checks challenges randomly with every call
def challenges_checker(df):
  for i in range(15):
    random_number = random.randint(1, len(df))
    print(list(df['29. State the 3 main challenges facing your county government.'].values)[random_number],'\n')

# Cleaning the data
cleaned_data = data.copy()

# Removing null values
cleaned_data.dropna(subset=['29. State the 3 main challenges facing your county government.'], inplace=True)

# The columns to be predicted
target_sentences = list(cleaned_data.columns)[1:]


# Concating function for challenges
def concating_func(x):
    if '\n' in x:
        challenges = x.split('\n')
        return [challenge.strip() for challenge in challenges if challenge.strip() != '']
    else:
        return [x]


cleaned_data['29. State the 3 main challenges facing your county government.'] = cleaned_data['29. State the 3 main challenges facing your county government.'].apply(concating_func)


display(cleaned_data.head())
cleaned_data.tail()

,29. State the 3 main challenges facing your county government.,Theme 1: Corruption/poor transparency/accountability/,Theme 2: Poor Civic Education/Public Participation/GRMs access to Information,Theme 3: Weak oversight/ Governance/procurement process,Theme 4: Conflicts between executive and county assembly,Theme 5: Poor legislative capacity,Theme 6: Weak finance management/poor leadership/procurement process,Theme 7: Climate issues/ insecurities Conflict,Theme 8: Human resource (HR) issues/ Nepotism/Cronyism/ tribalism,Theme 9: Poor service delivery/Lack of Digitization of services,"Theme 10: Inadequate resources, delayed treasury disbursements",Theme 11: Lack of opportunities /innovation/Investment unemployement/ Poverty/,Theme 12: Poor Inclusion/Gender/PWDs/Support to youths,Theme 13: Poor County systems/operations/ OSR collection,Theme 14: Poor InterGovernmental relations,Theme 15: Lack of political good will/ Political Influence/ Bad politics,Theme 16: Lack of Technical Capacity,Theme 17: Huge wage bill/Pending bills,Theme 18: Insecurity,"Theme 19: Poor Partnerships(Private Sector/NGOs), Poor Coordination"
0,"[No CBEF, No openness in procurement process b...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[1.Nepotism,corruption and favouritism in empl...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[Lack of guidelines on complaints and complime...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[Corruption, unemployment, poverty ]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[Water, waste management, wetlands encroachmen...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,29. State the 3 main challenges facing your county government.,Theme 1: Corruption/poor transparency/accountability/,Theme 2: Poor Civic Education/Public Participation/GRMs access to Information,Theme 3: Weak oversight/ Governance/procurement process,Theme 4: Conflicts between executive and county assembly,Theme 5: Poor legislative capacity,Theme 6: Weak finance management/poor leadership/procurement process,Theme 7: Climate issues/ insecurities Conflict,Theme 8: Human resource (HR) issues/ Nepotism/Cronyism/ tribalism,Theme 9: Poor service delivery/Lack of Digitization of services,"Theme 10: Inadequate resources, delayed treasury disbursements",Theme 11: Lack of opportunities /innovation/Investment unemployement/ Poverty/,Theme 12: Poor Inclusion/Gender/PWDs/Support to youths,Theme 13: Poor County systems/operations/ OSR collection,Theme 14: Poor InterGovernmental relations,Theme 15: Lack of political good will/ Political Influence/ Bad politics,Theme 16: Lack of Technical Capacity,Theme 17: Huge wage bill/Pending bills,Theme 18: Insecurity,"Theme 19: Poor Partnerships(Private Sector/NGOs), Poor Coordination"
352,[1. Not properly utilizing the available resou...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
353,"[Corruption, misplaced priorities, 2027 succes...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
354,"[Inadequate information, Corruption, Beuracracy]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
355,"[Corruption, theft, nepotism]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,[1. Lack of adequate information on county pro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
